# Tutorial 0: The Core of pykal

## Background

### Why did I make pykal?

Good question. After all, I am not an engineer. My background is in physics and mathematics; I hold a bachelors in the former and a masters in the latter. If you held a gun to my head and told me to engineer a paper bag, I would tell my family I love them and then tell you to pull the trigger. So it is strange that I would make a robotics framework; and truthfully, I wasn't drawn to robotics because of the money, or the status, or the hordes of women who would fall at my feet. Rather, I was drawn to the field by its progenitor: control theory. 

Control theory is at once elegant and powerful. If control theory is fundamental to all fields of engineering, then it is the field entire of robotics. I loved reading about estimation algorithms using Lie theory, or modeling drones using differential geometry, but I found, to my dismay, that there was no easy way to implement these ideas. Theory is beautiful, yes, but practice is validating, so why was there so little implementation of theory that promised results? 

After working in academia for some time, I discovered why: there was no standard way of getting ideas from "brain to bot". That is, the journey from head to hardware is long, messy, and often leaves in its wake a horrifying tangle of code and third-party software that no one but the author (and perhaps his research group) can use. And maybe not this time, but the next time, or the next time, or the time after that, when the original author has left or an outsider tries to follow his path, something will break and that's that. Of the control theorists I've met, many accept, suspect, or are haunted by this wasted effort, so it's no wonder most ideas stay safely on the page. 

This is a tragedy. This is unacceptable. This is why I made **pykal**, a portmanteau of "**py**thon" and "I can't believe it's this hard to implement a **kal**man filter in ROS". Like the child of a failing marriage, this framework was born out of frustration, and the unhappy coupling it sought to solve was that of theoretical control systems and practical robotics. But unlike the child of a failing marriage, I firmly believe **pykal** can help fix the current state of affairs.

So let's get started.


### The Feedback System

A basic feedback system[link] is shown below. 

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/system_input_observer_controller_block_diagram.svg" alt="System Observer Controller Block Diagram" style="width: 100%;">
   </div>

 We claim there are three objects in this block diagram which fall under the "control system" category:

- Dynamical System (or Plant)
- Observer (or Estimator)
- Controller

and that there are two objects which fall under the "function block" category:

- Signal generator
- t (Signal transform) 

As an intutive example, consider the cruise control in your car. The goal of cruise control is to maintain a set speed. The dynamical system is the car itself, where $y$ is the speedometer readings; the controller, observer, and signal generator/transforms are all parts of the car's CPU. The setpoint signal is generated by the CPU when you set the speed using a button or (if you're fancy) a touchscreen display; the difference between this setpoint and the current estimated speed is computed by a signal transform and sent to the controller; the controller looks at this difference and sends an appropriate signal to the car to either speed up or slow down (the controller abstracts away such things as gas pedal and brake pedal depression); the observer takes the controller signal and the speedometer's current readings and estimates what the current speed of the car is. 



### The Feedback System as Classes 

The control system objects are modeled by the following classes:

- Plant (`System`)
- Observer (`Observer`)
- Controller (`Controller`)
  
Note that in the feedback diagram above, it doesn't make sense to have an observer or a controller without an underlying system. To reflect this programmatically, the `System` class is injected as a dependency into the `Observer` and `Controller` classes. Below, we get a feel for constructing these objects; a thorough discussion on each class and its usage will be provided in later sections

In [ ]:
from pykal.control_system import System, Observer, Controller

sys = System(state_names=["x0","x1"]) # state names are the only required argument for the System constructor

obs = Observer(sys) # a System object must be passed into the Observer and Controller constructors 
cont = Controller(sys)

print(obs.sys) # the Observer and Controller objects reference the same System object in memory
print(cont.sys)

The function block objects are modeled by the following class:

- Signal Generator (`Signal`)
- Signal Transformer (`Signal`)  

Note that in the feedback diagram above, the signal generator and signal transformer do not depend explicitly on the control system objects; rather, they just define or modify the signals passed from one block to another. To reflect this programmatically, the `Signal` class has no dependencies. Below, we get a taste of what we can do with a `Signal` object; as before, a thorough discussion on its usage will be provided later. 

In [ ]:
from pykal.function_blocks import Signal

sig = Signal() # Signal constructor requires no arguments

sin_wave_func = sig.generate # generate signal functions
unit_step_func = sig.generate

# create a new signal function by applying a transform to two signal functions
sin_wave_func_times_unit_step_func = sig.transform_signals(sig.transforms.product,sin_wave_func,unit_step_func)


# bind this new signal function to the Signal object
sig.sin_wave_func_times_unit_step_func = sin_wave_func_times_unit_step_func

In the following sections, we will discuss the aforementioned classes in the `control_system` and `function_block` modules. For each class, the control theoretic conceptual primitive will be introduced and the mapping into each class will be explained. Basic usage of each class and important implementation details will also be discussed, with links provided for further reading when appropriate. 

## Control System

#### System
Recall that the state-space model of a continuous-time dynamical system is given by:

$$
\begin{aligned}
\dot{x} &= f(x, u, t) + w(t) \\
y &= h(x, u, t) + r(t)
\end{aligned}
$$

Where:

- $x \in \mathbb{R}^{n}$ is the **state vector**
- $u \in \mathbb{R}^{p}$ is the **input vector**
- $t \in \mathbb{R}^{+}$ is **time**
- $w(t) \in \mathbb{R}^{n}$ is **process noise**, distributed as $w \sim \mathcal{N}(0, Q(t))$, where $Q(t) \in \mathbb{R}^{n \times n}$ is the **process noise covariance matrix**

Similarly:

- $y \in \mathbb{R}^{m}$ is the **measurement vector**
- $r(t) \in \mathbb{R}^{m}$ is **measurement noise**, distributed as $r \sim \mathcal{N}(0, R(t))$, where $R(t) \in \mathbb{R}^{m \times m}$ is the **measurement noise covariance matrix**

In [ ]:
from pykal_core.control_system import System

sys = System(state_names=["x0","x1"]) # state names are the only required argument for the constructor

for key, value in sys.__dict__.items():
    print(f"{key}: {value}")

safeio: <pykal_core.control_system.system.System.SafeIO object at 0x7b5ac8953710>
_state_names: ['x0', 'x1']
_measurement_names: ['x0_meas', 'x1_meas']
_system_type: cti
_f: <function System.f_zero at 0x7b5ab1d316c0>
_h: <function System.h_identity at 0x7b5ab1d32700>
_Q: <function System.make_Q.<locals>.Q at 0x7b5ac8961120>
_R: <function System.make_R.<locals>.R at 0x7b5ac941a020>


Note that the `sys` object is initialized with several attributes if they are not passed explicity by the constructor (the `System.safeio` attribute is an instance of a utility subclass called `SafeIO` and is discussed here [link]). Each attribute is described briefly below:


sys.measurement_names
['x0_meas', 'x1_meas']

`sys.system_type` is the time structure of the dynamical system (default is "cti", or "continuous time-invariant")

sys.system_types # all available system time structures for sys
{'cti', 'ctv', 'dti', 'dtv'}

`System.f_zero` is the function $f(x,u,t) = 0$, that is, our plant has zero dynamics and does not change over time.

import numpy as np
xk = np.array([[1.0], [0.0]])
sys.f(xk)
array([[0.],
       [0.]])

`System.h_identity` is the function $h(x,u,t)=x$ ie the measurements are simply the state.

xk = np.array([[1.0], [0.0]])
sys.h(xk)
array([[1.],
       [0.]])

`System.make_Q.<locals>.Q` is the process noise covariance $Q$ created by a factory function `System.make_Q`. The default $Q$ returns an identity matrix with noise $0.01$ on the diagonals.       

sys.Q()
Note that the `sys` object is initialized with several attributes if they are not passed explicity by the constructor (the `System.safeio` object is an instance of utility subclass and will be discussed in the subsection `System.safeio`). Each attribute is described briefly below:

`sys._measurement_names` are the default measurement names (assumed to be full-state measurements) 

sys.measurement_names
['x0_meas', 'x1_meas']

`sys.system_type` is the time structure of the dynamical system (default is "cti", or "continuous time-invariant")

sys.system_types # all available system time structures for sys
{'cti', 'ctv', 'dti', 'dtv'}

`System.f_zero` is the function $f(x,u,t) = 0$, that is, our plant has zero dynamics and does not change over time.

import numpy as np
xk = np.array([[1.0], [0.0]])
sys.f(xk)
array([[0.],
       [0.]])

`System.h_identity` is the function $h(x,u,t)=x$ ie the measurements are simply the state.

xk = np.array([[1.0], [0.0]])
sys.h(xk)
array([[1.],
       [0.]])

`System.make_Q.<locals>.Q` is the process noise covariance $Q$ created by a factory function `System.make_Q`. The default $Q$ returns an identity matrix with noise $0.01$ on the diagonals.       

sys.Q()
array([[0.01, 0.  ],
       [0.  , 0.01]])

`System.make_R.<locals>.Q` is the measurement noise covariance $R$ created by a factory function `System.make_R`. The default $R$ returns an identity matrix with noise $0.01$ on the diagonals.

sys.R()
array([[0.01, 0.  ],
       [0.  , 0.01]])

['x0_meas', 'x1_meas']

{'cti', 'ctv', 'dti', 'dtv'}

array([[0.],
       [0.]])

`System.h_identity` is the function $h(x,u,t)=x$ ie the measurements are simply the state.

In [ ]:
xk = np.array([[1.0], [0.0]])
sys.h(xk)

array([[1.],
       [0.]])

In [ ]:
sys.Q()

array([[0.01, 0.  ],
       [0.  , 0.01]])

In [7]:
sys.R()

array([[0.01, 0.  ],
       [0.  , 0.01]])

As an example, consider the pendulum (pulled from tutorial 1) below:

In [ ]:
import numpy as np
from numpy.typing import NDArray

#### SafeIO

It seems that we can easily define system objects, as the previous example showed. But now we find ourselves in a subtle Pythonic pickle: what if our state dynamics only depended upon x, that is, ``f(x)`` instead of ``f(x,u,t)``? Any other classes or modules that compose System objects would expect ``System.f`` to have the arguments x, u, and t, in that order. We could simply define all functions like this, and choose not to use arguments that are not needed, but this breaks compatibability with existing tools. For instance, SciPy expects its functions in ``f(t, x)`` format, while JAX prefers ``f(x, u, t)`` or ``f(x, t)``. 

Furthermore, the flexibility of Python functions can be a double-edged sword. While it enables rapid prototyping, it also allows functions with inconsistent signatures, missing type annotations, ambiguous return types, or dynamic behavior that can silently break downstream logic.

Consider the pendulum example with C:

.. code-block:: c

   // C: fixed signature, explicit memory, compiler-enforced interface
   void dynamics(const double *x, const double *u, double t, double *x_dot_out) {
       x_dot_out[0] = x[1];
       x_dot_out[1] = -x[0];
   }

In C, functions must declare exactly what inputs they need and what outputs they produce.
Return types and pointer sizes are enforced at compile time, and passing the wrong number
of arguments or mismatched types raises immediate, traceable errors.

In Python, however, the same function might look like:

.. code-block:: python

   def f(x, u, t):
       return np.vstack([x[1], -x[0]])

We are at the mercy of runtime users to pass in the correct values, and to have the output be correct and of the correct shape. These kinds of mismatches are notoriously difficult to trace—especially when using external libraries like TensorFlow, PyTorch, or JAX that wrap or recompile functions dynamically.

Without a consistent interface, chaos creeps in. 

Hence, a utility class was created: ``SafeIO``.

This class validates user-defined functions at the time of registration (not just runtime). It also injects only necessary arguments and any keyword argumetns as needed, and enforces that all returned values are properly typed and shaped. It does this with the smart call function.

In [9]:
for key, value in sys.safeio.__dict__.items():
    print(f"{key}: {value}")

parent: <pykal_core.control_system.system.System object at 0x7b5ac8998920>
_aliases_for_x: ['x', 'x_k', 'xk', 'state']
_aliases_for_u: ['u', 'u_k', 'uk', 'input']
_aliases_for_t: ['t', 't_k', 'tk', 'time', 'tau']
smart_call: <bound method System.SafeIO.smart_call of <pykal_core.control_system.system.System.SafeIO object at 0x7b5ac8953710>>


In [ ]:
# 1. returns_scalar – should raise TypeError
try:
    def returns_scalar(x: NDArray) -> float:
        return 3.14

    x = np.array([[1.0]])
    sys.safeio.smart_call(returns_scalar, x=x)
except TypeError as e:
    print(f"TypeError: {e}")

TypeError: In `returns_scalar`, return type must be NDArray[...] or a tuple of NDArrays, but got <class 'float'>


In [ ]:
# 2. wrong_output_shape – should raise ValueError
try:
    def wrong_output_shape(x: NDArray) -> NDArray:
        return np.zeros((1, 1))

    sys.safeio.smart_call(wrong_output_shape, x=np.zeros((2, 1)), expected_shape=(2, 1))
except ValueError as e:
    print(f"ValueError: {e}")

ValueError: Output shape mismatch. Expected (2, 1), got (1, 1)


In [ ]:
# 3. no_arguments
def no_arguments() -> NDArray:
    return np.ones((2, 1))

print(sys.safeio.smart_call(no_arguments))

[[1.]
 [1.]]


In [ ]:
# 4. state_only
def state_only(x: NDArray) -> NDArray:
    return x + 1


x = np.array([[1.0], [2.0]])
print(sys.safeio.smart_call(state_only, x=x))

[[2.]
 [3.]]


In [ ]:
# 5. input_only
def input_only(u: NDArray) -> NDArray:
    return u * 2

u = np.array([[0.5], [1.0]])
print(sys.safeio.smart_call(input_only, u=u))

[[1.]
 [2.]]


In [ ]:
# 6. time_only
def time_only(t: float) -> NDArray:
    return np.array([[t], [t]])

print(sys.safeio.smart_call(time_only, t=3.0))

[[3.]
 [3.]]


In [ ]:
# 7. state_input
def state_input(x: NDArray, u: NDArray) -> NDArray:
    return x + u

print(sys.safeio.smart_call(state_input, x=x, u=u))

[[1.5]
 [3. ]]


In [ ]:
# 8. state_time
def state_time(x: NDArray, t: float) -> NDArray:
    return x * t

print(sys.safeio.smart_call(state_time, x=x, t=2.0))

[[2.]
 [4.]]


In [18]:
# 9. input_time
def input_time(u: NDArray, t: float) -> NDArray:
    return u + t

print(sys.safeio.smart_call(input_time, u=u, t=1.0))

[[1.5]
 [2. ]]


In [19]:
# 10. all_arguments
def all_arguments(x: NDArray, u: NDArray, t: float) -> NDArray:
    return x + u + t

print(sys.safeio.smart_call(all_arguments, x=x, u=u, t=1.0))

[[2.5]
 [4. ]]


In [20]:
# 11. reordered_arguments
def reordered_arguments(t: float, u: NDArray, x: NDArray) -> NDArray:
    return x + u + t

print(sys.safeio.smart_call(reordered_arguments, x=x, u=u, t=1.0))

[[2.5]
 [4. ]]


In [21]:
# 12. aliased_names
def aliased_names(state: NDArray, input: NDArray, time: float) -> NDArray:
    return state + input + time

print(sys.safeio.smart_call(aliased_names, x=x, u=u, t=1.0))

[[2.5]
 [4. ]]


In [22]:
# 13. with extra keyword arguments (note that u and t are not used either)
def just_x(x: NDArray) -> NDArray:
    return x * 2

print(sys.safeio.smart_call(just_x, x=x, u=u, t=2.0, extra_arg=3))

[[2.]
 [4.]]


In [23]:
def just_x_and_extra_kwarg(xk:NDArray,extra_arg:float) -> NDArray:
    return xk * extra_arg

print(sys.safeio.smart_call(just_x_and_extra_kwarg, x=x, u=u, t=2.0, extra_arg=3))

[[3.]
 [6.]]


### observer.py


#### Observer

Recall that the observer of a dynamical system is given by:

$$
\hat{x} = L(u,y)
$$

Where:
- $\hat{x} \in \mathbb{R}^{n}$ is the **estimated state vector**
- $y \in \mathbb{R}^{m}$ is the **measurement vector**
- $u \in \mathbb{R}^{p}$ is the **input vector**

Note that there is only one function that define the behavior of the observer: $L$. But this is high-level, often complexity comes in the functions composing L. For example, the kalman filter only requires y and u as active inputs, but is initialized with ..., and internal stuff.

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/observer_kf.svg" alt="Observer kf  Diagram" style="width: 50%;">
   </div>

In [24]:
from pykal_core.control_system.observer import Observer
obs = Observer(sys)

In [25]:
for key, value in obs.__dict__.items():
    print(f"{key}: {value}")

sys: <pykal_core.control_system.system.System object at 0x7b5ac8998920>
L: None


In [28]:
from pykal_core.utils.estimators import kf
obs.L = kf.make_L(obs,kf.predict_EKF,kf.update_EKF)

In [ ]:
obs.L.__name__

'kf'

For all estimators and how to use hhem, see the API reference here:

## function_blocks/